# 曲线拟合

## 目标和前提条件

尝试这个Jupyter笔记本建模示例，学习如何将函数拟合到一组观测值。我们将使用Gurobi Python API将这个回归问题表述为线性规划问题，然后用Gurobi优化器求解。

这个模型是H. Paul Williams的《数学规划中的模型构建》第五版第266页和第319-320页的示例11。

这个建模示例属于初级水平，我们假设你了解Python并且对构建数学优化模型有一定了解。读者还应该查阅
[文档](https://www.gurobi.com/resources/?category-filter=documentation)
了解Gurobi Python API。

**下载代码库** <br />
你可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 模型表述

### 集合和索引

$i \in \text{观测值}=\{1, .. ,n\}$.


### 参数

$x_{i} \in \mathbb{R}$: 观测值$i$处的自变量值。

$y_{i} \in \mathbb{R}$: 观测值$i$处的因变量值。

### 决策变量

$a \in \mathbb{R}$: 函数中解释$y$值与$x$值关系的常数项的值。

$b \in \mathbb{R}$: 函数中解释$y$值与$x$值关系的线性项系数。

$u_{i} \in \mathbb{R}^+$: 在观测值$i$处，所提议函数相对于$y$值的正偏差。

$v_{i} \in \mathbb{R}^+$: 在观测值$i$处，所提议函数相对于$y$值的负偏差。

$z$: 最大偏差值。

我们为第一个目标建模：

* 将直线$y=a+bx$拟合到给定的数据集，以最小化每个观测$y$值与线性关系预测值之间的绝对偏差之和。


### 问题1的约束

**偏差**: 每对对应的数据值$(x_{i},y_{i})$产生以下约束。

\begin{equation}
bx_{i} + a + u_{i} - v_{i} = y_{i}  \quad \forall i \in \text{观测值}
\end{equation}

其中$x_{i}$和$y_{i}$是观测集中给定的值，$b$、$a$、$u_{i}$和$v_{i}$是变量。
正偏差$u_{i}$和负偏差$v_{i}$给出了线性表达式建议的$y_{i}$值与观测值之间的差异量。

### 问题1的目标函数

**总偏差**: 目标是最小化总的正负偏差。

\begin{equation}
\text{最小化} \quad \sum_{i \in \text{观测值}} (u_{i} + v_{i})
\end{equation}

现在我们为第二个目标提供模型表述：

* 将直线$y=a+bx$拟合到给定的数据集，以最小化所有观测$y$值与线性关系预测值之间的最大偏差。

对于这个新表述，除了"偏差约束"外，我们还需要包含以下约束。

### 问题2的约束

**最大偏差**: 以下约束确保决策变量$z$取最大偏差值。

\begin{equation}
z \geq u_{i}  \quad \forall i \in \text{观测值}
\end{equation}

\begin{equation}
z \geq v_{i}  \quad \forall i \in \text{观测值}
\end{equation}

### 问题2的目标函数

**最小/最大偏差**: 目标是最小化最大偏差。

\begin{equation}
\text{最小化} \quad z
\end{equation}

## Python实现

我们导入Gurobi Python模块。

In [ ]:
# %pip install gurobipy

In [1]:
import gurobipy as gp
from gurobipy import GRB

# 使用Python 3.7.0和Gurobi 9.1.0测试

## 输入数据

我们定义观测集中$x$和$y$的对应值。

In [2]:
# 样本数据：自变量x和因变量y的值

observations, x, y = gp.multidict({
    ('1'): [0,1],
    ('2'): [0.5,0.9],
    ('3'): [1,0.7],
    ('4'): [1.5,1.5],
    ('5'): [1.9,2],
    ('6'): [2.5,2.4],
    ('7'): [3,3.2],
    ('8'): [3.5,2],
    ('9'): [4,2.7],
    ('10'): [4.5,3.5],
    ('11'): [5,1],
    ('12'): [5.5,4],
    ('13'): [6,3.6],
    ('14'): [6.6,2.7],
    ('15'): [7,5.7],
    ('16'): [7.6,4.6],
    ('17'): [8.5,6],
    ('18'): [9,6.8],
    ('19'): [10,7.3]
})

## 模型部署

我们创建一个模型和变量。模型的变量包括函数f(x)的常数项和线性项系数、正负偏差以及最大偏差。

In [7]:
model = gp.Model('CurveFitting')

# 函数f(x)的常数项。这是一个可以取正负值的自由连续变量。
a = model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="a")

# 函数f(x)的线性项系数。这是一个可以取正负值的自由连续变量。
b = model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="b")

# 捕获正偏差的非负连续变量
u = model.addVars(observations, vtype=GRB.CONTINUOUS, name="u")

# 捕获负偏差的非负连续变量
v = model.addVars(observations, vtype=GRB.CONTINUOUS, name="v")

# 捕获最大偏差值的非负连续变量
z = model.addVar(vtype=GRB.CONTINUOUS, name="z")

Set parameter LicenseID to value 2601452


每对对应的数据值$x_{i}$和$y_{i}$产生一个约束。

In [8]:
# 偏差约束

deviations = model.addConstrs( (b*x[i] + a + u[i] - v[i] == y[i] for i in observations), name='bias')

问题1的目标函数是最小化总的正负偏差。

In [9]:
# 问题1的目标函数

model.setObjective(u.sum('*') + v.sum('*'))

In [10]:
# 验证模型表述

# model.write('CurveFitting.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 19 rows, 41 columns and 75 nonzeros
Model fingerprint: 0x0bec2f7b
Coefficient statistics:
  Matrix range     [5e-01, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-01, 7e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.02s
Presolved: 19 rows, 40 columns, 75 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
      20    1.1466250e+01   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.146625000e+01


In [11]:
# 输出报告

print("\n\n_________________________________________________________________________________")
print(f"使偏差绝对值最小的最佳直线是：")
print("_________________________________________________________________________________")
print(f"y = {b.x:.4f}x + ({a.x:.4f})")



_________________________________________________________________________________
使偏差绝对值最小的最佳直线是：
_________________________________________________________________________________
y = 0.6375x + (0.5813)


对于问题2，需要引入另一个变量$z$来捕获最大偏差的值

In [12]:
# 最大偏差约束

maxPositive_deviation = model.addConstrs( (z >= u[i] for i in observations), name='maximum_positive_deviation')

maxNegative_deviation = model.addConstrs( (z >= v[i] for i in observations), name='maximum_negative_deviation')

问题2的目标函数是最小化最大偏差。

In [13]:
# 问题2的目标函数

model.setObjective(z)

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 57 rows, 41 columns and 151 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-01, 7e+00]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.146625e+01   0.000000e+00      0s
      11    1.7250000e+00   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.725000000e+00


In [14]:
# 输出报告

print("\n\n_________________________________________________________________________________")
print(f"使最大偏差最小的最佳直线是：")
print("_________________________________________________________________________________")
print(f"y = {b.x:.4f}x + ({a.x:.4f})")



_________________________________________________________________________________
使最大偏差最小的最佳直线是：
_________________________________________________________________________________
y = 0.6250x + (-0.4000)


---
## 参考文献

H. Paul Williams著，《数学规划中的模型构建》，第五版。

版权所有 © 2020 Gurobi Optimization, LLC